In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.linear_model import SGDRegressor, Ridge
from sklearn.metrics import r2_score

In [5]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [72]:
sgd = SGDRegressor(penalty='l2', alpha=0.01, max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)

print(r2_score(y_test, y_pred))
print(sgd.intercept_)
print(sgd.coef_)

0.2923564677875995
[151.47901206]
[ 33.6604731  -15.50133612 138.42648045 100.53491964  23.55958516
  11.24586329 -79.43918268  78.50634564 118.82908804  75.80346532]


In [73]:
ri = Ridge(alpha=0.2, solver='sparse_cg', max_iter=1000, random_state=42)
ri.fit(X_train, y_train)
y_pred = ri.predict(X_test)

print(r2_score(y_test, y_pred))
print(ri.intercept_)
print(ri.coef_)

0.4611298085258808
151.5905652735637
[  43.82817437 -178.13140292  463.74874187  295.70462409  -68.47726419
  -87.47929612 -191.88643375  143.27406849  353.63813498   94.95631673]


In [88]:
class MeraGDRidge:

    def __init__(self, alpha=0.01, epochs=1000, lr=0.01):
        self.intercept_ = None
        self.coef_ = None
        self.epochs = epochs
        self.lr = lr
        self.alpha = alpha

    def fit(self, X_train, y_train):
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        beta = np.insert(self.coef_, 0, self.intercept_)

        X_train = np.insert(X_train, 0, 1, axis=1)
        
        for epoch in range(self.epochs):
            gradient = (X_train.T @ X_train @ beta) - (X_train.T @ y_train) + (self.alpha * beta)
            beta = beta - (self.lr * gradient)

        self.intercept_ = beta[0]
        self.coef_ = beta[1:]

    def predict(self, X_test):
        return self.intercept_ + (X_test @ self.coef_)

In [89]:
mera_ri = MeraGDRidge(alpha=0.01, epochs=1000, lr=0.01)
mera_ri.fit(X_train, y_train)
y_pred = mera_ri.predict(X_test)

print(r2_score(y_test, y_pred))
print(mera_ri.intercept_)
print(mera_ri.coef_)

/tmp/ipykernel_11504/2670070848.py:18: RuntimeWarning: overflow encountered in matmul
  gradient = (X_train.T @ X_train @ beta) - (X_train.T @ y_train) + (self.alpha * beta)
/tmp/ipykernel_11504/2670070848.py:19: RuntimeWarning: invalid value encountered in subtract
  beta = beta - (self.lr * gradient)
/tmp/ipykernel_11504/2670070848.py:18: RuntimeWarning: invalid value encountered in matmul
  gradient = (X_train.T @ X_train @ beta) - (X_train.T @ y_train) + (self.alpha * beta)


ValueError: Input contains NaN.